# Exercise 1 - Simple Data Parallel Example

**GOAL:** The goal of this exercise is to show how to run simple tasks in parallel.

This script is too slow, and the computation is embarrassingly parallel. In this exercise, you will use Ray to execute the functions in parallel to speed it up.

### Concept for this Exercise - Remote Functions

The standard way to turn a Python function into a remote function is to add the `@ray.remote` decorator. Here is an example.

```python
# A regular Python function.
def regular_function():
    return 1

# A Ray remote function.
@ray.remote
def remote_function():
    return 1
```

The differences are the following:

1. **Invocation:** The regular version is called with `regular_function()`, whereas the remote version is called with `remote_function.remote()`.
2. **Return values:** `regular_function` immediately executes and returns `1`, whereas `remote_function` immediately returns an object ID (a future) and then creates a task that will be executed on a worker process. The result can be obtained with `ray.get`.
    ```python
    >>> regular_function()
    1
    
    >>> remote_function.remote()
    ObjectID(1c80d6937802cd7786ad25e50caf2f023c95e350)
    
    >>> ray.get(remote_function.remote())
    1
    ```
3. **Parallelism:** Invocations of `regular_function` happen **serially**, for example
    ```python
    # These happen serially.
    for _ in range(4):
        regular_function()
    ```
    whereas invocations of `remote_function` happen in **parallel**, for example
    ```python
    # These happen in parallel.
    for _ in range(4):
        remote_function.remote()
    ```

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import ray
import time

Start Ray. By default, Ray does not schedule more tasks concurrently than there are CPUs. This example requires four tasks to run concurrently, so we tell Ray that there are four CPUs. Usually this is not done and Ray computes the number of CPUs using `psutil.cpu_count()`. The argument `ignore_reinit_error=True` just ignores errors if the cell is run multiple times.

The call to `ray.init` starts a number of processes.

In [2]:
ray.init(num_cpus=4, ignore_reinit_error=True, include_webui=False)

2020-05-07 18:10:09,660	INFO resource_spec.py:205 -- Starting Ray with 7.52 GiB memory available for workers and up to 3.77 GiB for objects. You can adjust these settings with ray.remote(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '172.18.40.24',
 'redis_address': '172.18.40.24:57452',
 'object_store_address': '/tmp/ray/session_2020-05-07_18-10-09_652384_18687/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-07_18-10-09_652384_18687/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-05-07_18-10-09_652384_18687'}

**EXERCISE:** The function below is slow. Turn it into a remote function using the `@ray.remote` decorator.

In [4]:
# This function is a proxy for a more interesting and computationally
# intensive function.
def slow_function(i):
    time.sleep(1)
    return i

**EXERCISE:** The loop below takes too long. The four function calls could be executed in parallel. Instead of four seconds, it should only take one second. Once `slow_function` has been made a remote function, execute these four tasks in parallel by calling `slow_function.remote()`. Then obtain the results by calling `ray.get` on a list of the resulting object IDs.

In [ ]:
# Sleep a little to improve the accuracy of the timing measurements below.
# We do this because workers may still be starting up in the background.
time.sleep(2.0)
start_time = time.time()

results = [slow_function(i) for i in range(4)]

end_time = time.time()
duration = end_time - start_time

print('The results are {}. This took {} seconds. Run the next cell to see '
      'if the exercise was done correctly.'.format(results, duration))

**Solution**

In [10]:
@ray.remote
def slow_function(i):
    time.sleep(1)
    return i
start_time = time.time()
results = []
for i in range(4):
    results.append(slow_function.remote(i))
print(results)

[ObjectID(73962dd385f7ffffffff0100000000c001000000), ObjectID(26acd2812046ffffffff0100000000c001000000), ObjectID(41d260bc3921ffffffff0100000000c001000000), ObjectID(f3083a7ebdbaffffffff0100000000c001000000)]


In [11]:
results = ray.get(results)
print(results)
duration = time.time() - start_time
print('Executing the for loop took {:.3f} seconds.'.format(duration))
print('The results are:', results)
print('Run the next cell to check if the exercise was performed correctly.')

[0, 1, 2, 3]
Executing the for loop took 1.010 seconds.
The results are: [0, 1, 2, 3]
Run the next cell to check if the exercise was performed correctly.


**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [12]:
assert results == [0, 1, 2, 3], 'Did you remember to call ray.get?'
assert duration < 1.1, ('The loop took {} seconds. This is too slow.'
                        .format(duration))
assert duration > 1, ('The loop took {} seconds. This is too fast.'
                      .format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 1.0102510452270508 seconds.


**EXERCISE:** Use the UI to view the task timeline and to verify that the four tasks were executed in parallel. You can do this as follows.

1. Run the following cell to generate a JSON file containing the profiling data.
2. Download the timeline file by right clicking on `timeline01.json` in the navigator to the left and choosing **"Download"**.
3. Open [chrome://tracing/](chrome://tracing/) in the Chrome web browser, click on the **"Load"** button and load the downloaded JSON file.

To navigate within the timeline, do the following.
- Move around by clicking and dragging.
- Zoom in and out by holding **alt** and scrolling.

**NOTE:** The timeline visualization will only work in **Chrome**.

In [14]:
ray.timeline(filename="timeline01.json")

download the json and open from [chrome://tracing](chrome://tracing)